## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-15-13-08-37 +0000,nyt,4 More Lawmakers in Video Say Federal Prosecut...,https://www.nytimes.com/2026/01/14/us/politics...
1,2026-01-15-13-07-49 +0000,nyt,Iran Says It Will Not Execute Protester Amid I...,https://www.nytimes.com/2026/01/15/world/middl...
2,2026-01-15-13-01-47 +0000,nyt,Thailand Has 2nd Deadly Accident Involving a C...,https://www.nytimes.com/2026/01/15/world/asia/...
3,2026-01-15-13-01-19 +0000,bbc,Millions of people to see their local council ...,https://www.bbc.com/news/articles/cd7z892jzdeo...
4,2026-01-15-12-57-01 +0000,nyt,"After an Inconclusive Meeting, Greenlanders As...",https://www.nytimes.com/2026/01/15/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2500/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,66
5,iran,37
81,new,20
312,ice,18
71,minneapolis,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
219,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...,143
117,2026-01-15-08-35-55 +0000,wsj,Stock Market Today: Nasdaq Futures Rise; Oil S...,https://www.wsj.com/livecoverage/stock-market-...,139
191,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...,136
285,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...,135
337,2026-01-14-17-25-00 +0000,nypost,Iran issues sickening assassination threat aga...,https://nypost.com/2026/01/14/world-news/iran-...,134


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
219,143,2026-01-14-22-56-00 +0000,wsj,Top officials from Denmark and Greenland said ...,https://www.wsj.com/politics/national-security...
169,74,2026-01-15-02-13-51 +0000,cbc,Video shows woman dragged from car by ICE agen...,https://www.cbc.ca/news/world/ice-woman-car-mi...
117,73,2026-01-15-08-35-55 +0000,wsj,Stock Market Today: Nasdaq Futures Rise; Oil S...,https://www.wsj.com/livecoverage/stock-market-...
271,66,2026-01-14-20-50-09 +0000,bbc,US launches phase two of Gaza peace plan with ...,https://www.bbc.com/news/articles/cjw1nxe5pvlo...
191,64,2026-01-15-00-54-00 +0000,wsj,The Senate voted to kill a war-powers resoluti...,https://www.wsj.com/politics/policy/trump-beat...
285,58,2026-01-14-20-17-00 +0000,wsj,"Fed governor Stephen Miran, an appointee of Pr...",https://www.wsj.com/economy/central-banking/fe...
275,48,2026-01-14-20-36-08 +0000,nypost,Trump says ‘other side’ claims it stopped kill...,https://nypost.com/2026/01/14/us-news/trump-sa...
317,43,2026-01-14-18-27-54 +0000,latimes,Child porn concerns prompt California to launc...,https://www.latimes.com/california/story/2026-...
27,37,2026-01-15-12-16-30 +0000,nypost,Mayor Mamdani’s radical tenant advocate Cea We...,https://nypost.com/2026/01/15/us-news/mayor-ma...
322,37,2026-01-14-18-10-00 +0000,wsj,As Washington goes on offense against global c...,https://www.wsj.com/articles/lawmakers-to-rest...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
